# For my Safety reason, load setting file from tests folder

In [2]:
import os, re,json
import pandas as pd

first_wd = os.getcwd()
print('First Directory at :', os.getcwd()[2:])

os.chdir(re.sub(os.getcwd().split('\\')[-1]+'$','',os.getcwd()) + 'tests')
print('Change to :', os.getcwd()[2:])
#Load Setting File
with open('setting.txt','r') as f:
    setting = json.load(f)
print('Load Setting OK') 

os.chdir(first_wd)
print('Change back to :', os.getcwd()[2:])

First Directory at : \01_DEV_Function\Shared_Function\samples
Change to : \01_DEV_Function\Shared_Function\tests
Load Setting OK
Change back to : \01_DEV_Function\Shared_Function\samples


# Working with da_tran_SQL 

In [4]:
from py_topping.data_connection.database import da_tran_SQL

## SQL Server (or MS SQL)

In [3]:
set_mssql = setting['MSSQL']
print(set_mssql['type'])

MSSQL


### Create connection to server

In [4]:
mssql = da_tran_SQL(sql_type = set_mssql['type'],
                host_name = set_mssql['host'],
                database_name = set_mssql['database'],
                user = set_mssql['user'],
                password = set_mssql['password'])

Connection OK


for SQL Server the <b>sql_type must == "MSSQL" (ignore case sensitive in version >= 0.1.3)</b>
  
    
by Default, the class will using pymssql as driver and port 1433
    
But you could also change them if you want like below

In [5]:
mssql = da_tran_SQL(sql_type = set_mssql['type'],
                host_name = set_mssql['host'],
                database_name = set_mssql['database'],
                user = set_mssql['user'],
                password = set_mssql['password'],
                port = '1433',
                driver = 'pymssql')

Connection OK


The da_tran_SQL class also has engine method which equal to engine in sqlalchemy

Which I'm going to use it to set up sample environment that include,
 * 2 dataframe
 * table in mssql
 * store procedure in mssql

In [6]:
def setup_sample(class_in):
    test_table = 'unit_test_git'
    df1 = pd.DataFrame({'col1' : [1,2,3,4,5] , 'col2' : [1,1,2,2,3] 
                        ,'date' : pd.date_range('2020-10-10','2020-10-14') , 'col3' : [1,1,1,1,1]})
    df2 = pd.DataFrame({'col1' : [4,5,6,7,8] , 'col2' : [3,3,4,4,5] 
                        ,'date' : pd.date_range('2020-10-12','2020-10-16') , 'col3' : [2,2,2,2,2]})
    df1.to_sql(test_table, index = False, if_exists = 'replace', con = mssql.engine)

    sql_q = """ CREATE OR ALTER PROCEDURE unit_test_git_SP (@PARAM1 AS VARCHAR(100))
                AS
                BEGIN
                SELECT * FROM {} WHERE col1 = @PARAM1 
                END""".format(test_table)

    class_in.engine.execute(sql_q)

    return df1, df2, test_table, 'unit_test_git_SP'

df1, df2, tb, sp = setup_sample(mssql)
pd.read_sql('unit_test_git', con = mssql.engine)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


### 1st Method : Read

You could read any table or view from your database by use method "read"

In [7]:
mssql.read(tb)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


You could also use pass SQL's "WHERE" statement into parameter "condition_in"

to filter the result

In [8]:
mssql.read(tb, condition_in = 'col1 <= 2')

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1


futher more, you could also use store procedure and pass dict parameter with read method

In [9]:
mssql.read(sp, SP = True, param = {'@PARAM1' : 3})

,col1,col2,date,col3
0,3,2,2020-10-12,1


You could select only unique value of selected column

In [10]:
# Coming Soon...

You could group by and use aggregrate function 

In [11]:
# Coming Soon...

### 2nd Method : dump_whole 

You could replace whole table with new dataframe with is method

<font color='red'>Becareful with Columns' Type, I'm suggest to fix columns' type before using this method </font>

In [12]:
df1, df2, tb, sp = setup_sample(mssql)
mssql.read(tb)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [13]:
mssql.dump_whole(df2, tb)

mssql.read(tb)

Start Filter Existing data from df at  2020-12-07 16:32:14.780796
Dump data to  unit_test_git  End  2020-12-07 16:32:15.322504


,col1,col2,date,col3
0,4,3,2020-10-12,2
1,5,3,2020-10-13,2
2,6,4,2020-10-14,2
3,7,4,2020-10-15,2
4,8,5,2020-10-16,2


### 3rd Method : dump_replace

You could update the existing dataframe by using this method

In [14]:
df1, df2, tb, sp = setup_sample(mssql)
mssql.read(tb)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [15]:
df2

,col1,col2,date,col3
0,4,3,2020-10-12,2
1,5,3,2020-10-13,2
2,6,4,2020-10-14,2
3,7,4,2020-10-15,2
4,8,5,2020-10-16,2


If col1 is the primary key for table and df2

we could update the existing key and add new key into table by using dump_replace 

In [16]:
mssql.dump_replace(df2, tb, list_key = ['col1'])
mssql.read(tb)

Start delete old data at 2020-12-07 16:32:16.190836
Delete Last ['col1'] at 2020-12-07 16:32:16.276459
Dump data to  unit_test_git  End  2020-12-07 16:32:16.421067


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,3,2020-10-12,2
4,5,3,2020-10-13,2
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


For table with multiple keys, we could just simple add more key into the parameter "list_key"

In [17]:
df1, df2, tb, sp = setup_sample(mssql)
mssql.read(tb)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [18]:
mssql.dump_replace(df2, tb, list_key = ['col1','col2'])
mssql.read(tb)

Start delete old data at 2020-12-07 16:32:17.249637
Delete Last ['col1', 'col2'] at 2020-12-07 16:32:17.331459
Dump data to  unit_test_git  End  2020-12-07 16:32:17.469107


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,4,3,2020-10-12,2
5,5,3,2020-10-13,2
6,6,4,2020-10-14,2
7,7,4,2020-10-15,2
8,8,5,2020-10-16,2


dump_replace also has math logic for some ETL process

In [19]:
df1, df2, tb, sp = setup_sample(mssql)

In [20]:
mssql.dump_replace(df2, tb, list_key = ['date'], 
                   math_logic = {'date' : {'logic' : '>=', 'value' : df2['date'].min().date(), 'type' : 'date'}} )

mssql.read(tb)

Start delete old data at 2020-12-07 16:32:18.201008
Delete Last ['date'] at 2020-12-07 16:32:18.289947
Dump data to  unit_test_git  End  2020-12-07 16:32:18.462276


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,4,3,2020-10-12,2
3,5,3,2020-10-13,2
4,6,4,2020-10-14,2
5,7,4,2020-10-15,2
6,8,5,2020-10-16,2


 math logic also could use with num too

In [21]:
df1, df2, tb, sp = setup_sample(mssql)

In [22]:
mssql.dump_replace(df2, tb, list_key = ['col1','date'], 
                   math_logic = {'col1' : {'logic' : '>=', 'value' : df2['col1'].min(), 'type' : 'int'} ,
                                 'date' : {'logic' : '>=', 'value' : df2['date'].min().date(), 'type' : 'date'}} )

mssql.read(tb)

Start delete old data at 2020-12-07 16:32:19.256544
Delete Last ['col1', 'date'] at 2020-12-07 16:32:19.336450
Dump data to  unit_test_git  End  2020-12-07 16:32:19.481182


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,3,2020-10-12,2
4,5,3,2020-10-13,2
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


You could also combine math_logic and non logic key together

In [23]:
df1, df2, tb, sp = setup_sample(mssql)

In [24]:
mssql.dump_replace(df2, tb, list_key = ['col1','col2'], 
                   math_logic = {'col1' : {'logic' : '>', 'value' : df2['col1'].min(), 'type' : 'int'} } )

mssql.read(tb)

Start delete old data at 2020-12-07 16:32:20.250690
Delete Last ['col1', 'col2'] at 2020-12-07 16:32:20.342339
Dump data to  unit_test_git  End  2020-12-07 16:32:20.489196


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,4,3,2020-10-12,2
5,5,3,2020-10-13,2
6,6,4,2020-10-14,2
7,7,4,2020-10-15,2
8,8,5,2020-10-16,2


Since there're many things going inside this function

This Function also have boolean paramter "debug" to print what's going on inside

In [25]:
mssql.dump_replace(df2, tb, list_key = ['col1','date'], 
                   math_logic = {'col1' : {'logic' : '>=', 'value' : df2['col1'].min(), 'type' : 'int'} ,
                                 'date' : {'logic' : '>=', 'value' : df2['date'].min().date(), 'type' : 'date'}}
                  ,debug = True)

mssql.read(tb)

Start delete old data at 2020-12-07 16:32:20.549220
Delete Last ['col1', 'date'] at 2020-12-07 16:32:20.633297
delete from unit_test_git where [col1] >= 4 and [date] >= '2020-10-12'
Dump data to  unit_test_git  End  2020-12-07 16:32:20.767764


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,3,2020-10-12,2
4,5,3,2020-10-13,2
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


To be honest, I'd recommend you to use a text and good defined key.

***

### 4th Method : dump_new

dump only non existing key into the table

In [26]:
df1, df2, tb, sp = setup_sample(mssql)

mssql.read(tb)

,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1


In [27]:
mssql.dump_new(df2, tb, list_key = ['col1'])

mssql.read(tb)

Start Filter Existing data from df at  2020-12-07 16:32:21.531746
Dump data to  unit_test_git  End  2020-12-07 16:32:21.732196


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1
5,6,4,2020-10-14,2
6,7,4,2020-10-15,2
7,8,5,2020-10-16,2


Just like dump_replace, you colud also use multiple key at once.

In [28]:
df1, df2, tb, sp = setup_sample(mssql)

In [29]:
mssql.dump_new(df2, tb, list_key = ['col1','col2'])

mssql.read(tb)

Start Filter Existing data from df at  2020-12-07 16:32:22.473215
Dump data to  unit_test_git  End  2020-12-07 16:32:22.665463


,col1,col2,date,col3
0,1,1,2020-10-10,1
1,2,1,2020-10-11,1
2,3,2,2020-10-12,1
3,4,2,2020-10-13,1
4,5,3,2020-10-14,1
5,4,3,2020-10-12,2
6,6,4,2020-10-14,2
7,7,4,2020-10-15,2
8,8,5,2020-10-16,2


# MySQL

In [6]:
set_mysql = setting['MYSQL']
print(set_mysql['type'])

MYSQL


### Create connection to server

In [9]:
mysql = da_tran_SQL(sql_type = set_mysql['type'],
                host_name = set_mysql['host'],
                database_name = set_mysql['database'],
                user = set_mysql['user'],
                password = set_mysql['password'])

Connection OK


Same as SQL Server, For MySQL the <b>sql_type must == "MYSQL" (ignore case sensitive)</b>
<br><br>
by Default, the class will using pymysql as driver and port 3306
<br><br>
But you could also change them if you want like below

In [10]:
mysql = da_tran_SQL(sql_type = set_mysql['type'],
                host_name = set_mysql['host'],
                database_name = set_mysql['database'],
                user = set_mysql['user'],
                password = set_mysql['password'],
                port = '3306',
                driver = 'pymysql')

Connection OK


<b>Every Method and Function are the same as SQL Server.</b>

# PostGreSQL

In [5]:
set_plsql = setting['POSTGRESQL']
print(set_plsql['type'])

POSTGRESQL


### Create connection to server

In [6]:
plsql = da_tran_SQL(sql_type = set_plsql['type'],
                host_name = set_plsql['host'],
                database_name = set_plsql['database'],
                user = set_plsql['user'],
                password = set_plsql['password'])

Connection OK


Same as SQL Server, For MySQL the <b>sql_type must == "MYSQL" (ignore case sensitive)</b>
<br><br>
by Default, the class will using pymysql as driver and port 3306
<br><br>
But you could also change them if you want like below

In [9]:
plsql = da_tran_SQL(sql_type = set_plsql['type'],
                host_name = set_plsql['host'],
                database_name = set_plsql['database'],
                user = set_plsql['user'],
                password = set_plsql['password'],
                port = '5432',
                driver = 'psycopg2')

Connection OK


<b>Every Method and Function are the same as SQL Server.<br>
Except PostGreSQL still can't use Store Procedure or Function yet</b>

# OracleSQL

In [32]:
# Coming Soon...